# Cluster the Spire
Will Wright

### Purpose and Context

[todo]

In [3]:
# Load packages
import shutil
from os import listdir
import json
import glob
import os
import numpy as np
import pandas as pd
import random
import copy
from heapq import nsmallest

# increase viewable dataframe rows and columns
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 20)

# set random seed
random.seed(30)

All the data currently lives in several zipped tar.gz files within the 'zipped' folder.  These need to be extracted into an unzipped folder.

**PROTIP:** If you have the files already extracted (as they are in the repo), skip this step to avoid the lengthy unpacking process

In [2]:
def extract_all(archives, extract_path, zip_format = "gztar"):
    '''
    input: path to zipped file archives, path to extract, and type of zipped file
    output: unzipped contents of each zipped file within the extract path
    '''
    for filename in listdir(archives):
        shutil.unpack_archive(archives+filename, extract_path, zip_format)

In [32]:
extract_all("../data_raw/zipped/","../data_raw/unzipped/", "gztar")

In [3]:
# Start here if the files are already unzipped
read_files = glob.glob("../data_raw/unzipped/*/*.json", recursive = True)

To give more context about the data we're working with, lets see exactly how many raw game runs we have:

In [4]:
len(read_files)

279848

Almost 280K games! We'll need to subset down to games for The Defect on Ascension 20 that resulted in wins before we can determine the relevant sample size though. In order to do that, we'll want to read these files together and use relevant JSON keys to narrow our focus.

In [5]:
# this approach creates a list of JSON strings from all the read_files
output_list = []

for f in read_files:
    try:
        with open(f, "r") as infile:
            # test if the file isn't empty and that the name doesn't contain 'undefined' (1 file, contents are "File doesn't exists)")
            if (os.path.getsize(f)>0) & (('undefined' in f)==False):
                output_list.append(json.load(infile))
            else:
                pass
    except UnicodeDecodeError: # some unicode can't be read so just don't load those games (I think it's a particular monster name)
        pass
    

In [6]:
len(output_list)

279693

In [7]:
len(read_files)-len(output_list)

155

We've excluded 155 games that were either empty, had unreadable unicode, or were 'undefined'.  It's possible that this may introduce some bias (e.g. removing relevant games with particular qualities), but given that this represents such a small volume of games relative to all 280K and I haven't seen any apparent bias in looking through a sample of the files, I don't think this should be a major concern.

After more attempts to get the data into the right format, it looks like there is a single case where the JSON is wrapped in '[ ]'.  Since this game is for Ironclad, I'll simply remove from the dataset.

In [8]:
len(output_list)

279693

In [9]:
output_list[:] = [s for s in output_list if str(s)[0]!='[']

In [10]:
len(output_list)

279692

With that single exception removed, we can now subset to a list of Defect games, which pass the conditions of being the Defect character, a victory, and Ascension 20.  Since it's possible that I'll want to expand this investigation to the other two characters later, I'll also set aside their games in their own lists.

In [11]:
# Winning Ascension 20 games by character
defect_asc20_win_games = []
ironclad_asc20_win_games = []
silent_asc20_win_games = []

# Losing Ascension 20 games by character
defect_asc20_lose_games = []
ironclad_asc20_lose_games = []
silent_asc20_lose_games = []

for i in range(len(output_list)):
    if output_list[i] is not None:
        # test to ensure the game data has all the required elements (character, ascention level, and victory status)
        if ('character_chosen' in dict(output_list[i])) and \
        ('ascension_level' in dict(output_list[i])) and \
        ('victory' in dict(output_list[i])):
            
            # DEFECT WINNING
            if (output_list[i]['character_chosen']=='DEFECT') & \
            (output_list[i]['victory']==True) & \
            (output_list[i]['ascension_level']==20):
                defect_asc20_win_games.append(output_list[i])
            
            # DEFECT LOSING
            if (output_list[i]['character_chosen']=='DEFECT') & \
            (output_list[i]['victory']==False) & \
            (output_list[i]['ascension_level']==20):
                defect_asc20_lose_games.append(output_list[i])
            
            # IRONCLAD WINNING  
            if (output_list[i]['character_chosen']=='IRONCLAD') & \
            (output_list[i]['victory']==True) & \
            (output_list[i]['ascension_level']==20):
                ironclad_asc20_win_games.append(output_list[i])
                
            # IRONCLAD LOSING  
            if (output_list[i]['character_chosen']=='IRONCLAD') & \
            (output_list[i]['victory']==False) & \
            (output_list[i]['ascension_level']==20):
                ironclad_asc20_lose_games.append(output_list[i])
                
            # SILENT WINNING  
            if (output_list[i]['character_chosen']=='THE_SILENT') & \
            (output_list[i]['victory']==True) & \
            (output_list[i]['ascension_level']==20):
                silent_asc20_win_games.append(output_list[i])
                
            # SILENT LOSING
            if (output_list[i]['character_chosen']=='THE_SILENT') & \
            (output_list[i]['victory']==False) & \
            (output_list[i]['ascension_level']==20):
                silent_asc20_lose_games.append(output_list[i])

I'm curious about character winrates.  Lets compare to the total games per character.

In [12]:
# Calculate all summary statistics
defect_winning = len(defect_asc20_win_games)
defect_losing = len(defect_asc20_lose_games)
defect_total = len(defect_asc20_win_games)+len(defect_asc20_lose_games)
defect_winrate = defect_winning/defect_total

# Calculate all summary statistics
ironclad_winning = len(ironclad_asc20_win_games)
ironclad_losing = len(ironclad_asc20_lose_games)
ironclad_total = len(ironclad_asc20_win_games)+len(ironclad_asc20_lose_games)
ironclad_winrate = ironclad_winning/ironclad_total

# Calculate all summary statistics
silent_winning = len(silent_asc20_win_games)
silent_losing = len(silent_asc20_lose_games)
silent_total = len(silent_asc20_win_games)+len(silent_asc20_lose_games)
silent_winrate = silent_winning/silent_total


asc20_games_summary = pd.DataFrame({'Character':['Defect','Ironclad','Silent'],
                                    'Winning Games':[defect_winning,
                                                     ironclad_winning,
                                                     silent_winning],
                                     'Total Games':[defect_total,
                                                    ironclad_total,
                                                    silent_total],
                                     'Winrate':[defect_winrate,
                                               ironclad_winrate,
                                               silent_winrate]})



In [13]:
asc20_games_summary

,Character,Winning Games,Total Games,Winrate
0,Defect,1669,16863,0.098974
1,Ironclad,1716,14798,0.115962
2,Silent,1811,14278,0.126838


It would seem that although Defect is the most-played character, it has the lowest winrate.  This supports the claim that Defect is the hardest character (at least on Ascension 20).  In any case, we have 1669 victorious Defect Ascension 20 games, which should be adequate sample for clustering.

Next, we need to convert this list of JSON objects to a dataframe we can cluster.  Ideally, the shape of the data is one-row-per-game with columns for all the cards and relics. In order to do that, we'll want to create a vector of all unique cards and relics.  

#### Getting Unique Cards and Relics  

In order to get all unique cards and relics, we can simply pull all cards and relics from all games, then apply the `unique()` function.

In [14]:
all_game_decks = []
all_game_relics = []

for i in range(len(output_list)):
    if output_list[i] is not None:
        # ensure the run data has the deck and relics to avoid errors in rare cases
        if ('master_deck' in dict(output_list[i])) and ('relics' in dict(output_list[i])):
            all_game_decks.append(output_list[i]['master_deck'])
            all_game_relics.append(output_list[i]['relics'])

In [15]:
# Within each game, each card and relic needs to be pulled out into a flat list.
all_cards = []

for i in range(len(all_game_decks)):
    for j in range(len(all_game_decks[i])):
        all_cards.append(all_game_decks[i][j])
        
all_relics = []

for i in range(len(all_game_relics)):
    for j in range(len(all_game_relics[i])):
        all_relics.append(all_game_relics[i][j])


In [16]:
# create unique lists
unique_cards = list(np.unique(all_cards))
unique_relics = list(np.unique(all_relics))

In [18]:
len(unique_cards)

3164

In [19]:
len(unique_relics)

876

Looks like we have 3164 unique cards and 876 unique relics.  This is a fair bit more than expected, so lets take a look at the head and tail of cards:

In [20]:
unique_cards[0:10]

['6A',
 '6A+1',
 'A Thousand Cuts',
 'A Thousand Cuts+1',
 'Abandon',
 'Abandon+1',
 'AbeCurse',
 'AbsoluteMagnitude+1',
 'Absolvement',
 'Absolvement+1']

In [21]:
unique_cards[-10:-1]

['vexMod:StarBlast',
 'vexMod:StrikeStorm',
 'vexMod:StrikeStorm+1',
 'vexMod:Taunt+1',
 'vexMod:TrainingStrike',
 'vexMod:TrainingStrike+1',
 'vexMod:UltimateCard',
 'vexMod:VenomSigh',
 'vexMod:VolumeVengeance']

This reveals two issues: there are the standard and "+1" versions of cards (players can upgrade cards once) as well as cards from game mods (essentially, player-made extensions of the game).  Thankfully, my domain expertise makes it fairly easy to know which cards aren't in the base game and it seems like most of the modded cards have a ':' in their name so they should be fairly easy to exclude.  

After testing, it looks like there are a few other exceptions for specific mods that use a '\_' in their name.  I'll go ahead and simply remove those cases as well.

In [22]:
unique_cards[:] = [s for s in unique_cards if '+' not in s \
                   and ':' not in s\
                   and '_' not in s]

In [23]:
len(unique_cards)

679

In review of the new card list, I can still see some non-base cards, but I'm not too concerned with this affecting the final results due to the expected low frequency of those cards (0 in cases where the character isn't one of the base characters).  

Next, the same cleansing will be applied to the relics. Generally speaking, relics have the same issue with mods as the cards, but there are not upgrades available.

In [24]:
unique_relics[:] = [s for s in unique_relics if '_' not in s \
                   and ':' not in s]

In [25]:
len(unique_relics)

391

Again, this isn't a perfect methodology, but since there are no flags for being a mod within the game data, it is difficult to use a single signal as a subsetting criteria to only the base game.

___

At this point, we can build a table of all unique cards and relics and then fill in Trues and Falses for whether the card was present per completed game.

In [56]:
def resource_table_generator(resource_input, game_input):
    '''
    input: a list of resources to be included in the rows of the table (i.e. cards and relics)
           a list of JSON game data to go in the columns
    output: a DataFrame indicating if each resource is in each game with a True or False
    '''
    # Build a 1-column table with the resources in each row
    resource_table = pd.DataFrame({'Resource':resource_input})
    
    # determine the number of preceding zeroes to put in the game name for the column names
    game_volume = len(game_input)
    preceding_zeroes = len(str(game_volume)) # so if you have 1669 games, the first game will be 'game_0001'
    
    # For each game append a column and fill with True/False based on if the card is present
    for i in range(len(game_input)):
        # scrub the '+1's from the cards (just remove the string, not the card) so they'll match on the unique_cards list
        game_deck = game_input[i]['master_deck']
        for j in range(len(game_deck)):
            game_deck[j] = game_deck[j].replace('+1','')
        
        # Create column name for the game
        resource_table['game_'+str('{:0'+str(preceding_zeroes)+'d}').format(i+1)] = False # default to false
        # For each resource, update to True if it is in the final game resources
        for j in range(len(resource_table)):
            if((resource_table['Resource'][j] in game_deck) or \
               (resource_table['Resource'][j] in game_input[i]['relics'])):
                resource_table.iloc[j,i+1] = True
    
    return(resource_table)


#### Defect Resource Table and Summary Frequency Tables

Now, simply plug in the resources and the Defect Ascension 20 victorious games to get a complete table of whether each resource is present in each game.

In [57]:
defect_asc20_win_resources = resource_table_generator(unique_cards+unique_relics, defect_asc20_win_games)

In [58]:
defect_asc20_win_resources

,Resource,game_0001,game_0002,game_0003,game_0004,game_0005,game_0006,game_0007,game_0008,game_0009,...,game_1660,game_1661,game_1662,game_1663,game_1664,game_1665,game_1666,game_1667,game_1668,game_1669
0,6A,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,A Thousand Cuts,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,Abandon,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,AbeCurse,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Absolvement,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1065,Winged Necklace,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1066,WingedGreaves,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1067,WristBlade,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1068,Yang,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


It'd be useful to see the relative frequency of each resource in a separate summary table:

In [59]:
defect_resource_freq = pd.DataFrame({'Resource':defect_asc20_win_resources['Resource'],
                                     'Frequency':defect_asc20_win_resources.sum(axis = 1)})
defect_resource_freq = defect_resource_freq.sort_values(by = ['Frequency'], ascending = False).reset_index(drop=True)
defect_resource_freq['Percent of Wins'] = defect_resource_freq['Frequency']/len(defect_asc20_win_games)

In [60]:
# subset to resources which are >0
defect_resource_freq = defect_resource_freq[defect_resource_freq['Frequency']>0]
defect_resource_freq['Percentile Rank']=defect_resource_freq['Frequency'].rank(pct=True, ascending = False)
defect_resource_freq['Rank']=defect_resource_freq['Frequency'].rank(ascending = False)

In [76]:
defect_resource_freq[0:10]

,Resource,Frequency,Percent of Wins,Percentile Rank,Rank,Resource Type
0,AscendersBane,1646,0.986219,0.002488,1.0,card
1,Dualcast,1513,0.906531,0.004975,2.0,card
2,Zap,1401,0.839425,0.007463,3.0,card
3,Cracked Core,1394,0.835231,0.009950,4.0,relic
4,Coolheaded,1357,0.813062,0.012438,5.0,card
5,Defragment,1163,0.696824,0.014925,6.0,card
6,Hologram,1141,0.683643,0.017413,7.0,card
7,Glacier,1039,0.622528,0.019900,8.0,card
8,Cold Snap,931,0.557819,0.022388,9.0,card
9,Capacitor,889,0.532654,0.024876,10.0,card


These results align with expectations.  'AscendersBane' is the curse card which should be present in every winning deck with Defect on Ascension 20 and thus, it makes sense that it's the most frequent card (though I'm not quire sure how some games completed without it).  Below that are the starting cards 'Dualcast' and 'Zap', then the starting relic 'Cracked Core'.  The first non-starting resource is 'Coolheaded' and it appears in a full **81%** of games! I knew it was a good card, but it seems almost critical to success on Ascension 20 with Defect.

After having used this .csv when playing, it seems like it'd be more user-friendly to have a total of 3 .csvs per character.  We've created one (a view containing all relics and cards), but if I only wanted to select cards in the top 20th percentile, I can't because the relics are mixed in.  This being the case, we'll separate into cards and relics, then get the percentiles and ranks within those groupings.  

To do this, we'll use a function to assign 'card' or 'relic' to `defect_resource_freq` in a 'resource_type' column, then subset on that before performing the ranking.

In [64]:
def resource_typer(resource_name_input):
    '''
    input: name of a resource
    output: string of 'relic' or 'card' based on the input
    '''
    if resource_name_input in unique_cards: return('card')
    if resource_name_input in unique_relics: return('relic')

In [73]:
defect_resource_freq['Resource Type']=''
for i in range(len(defect_resource_freq)):
    defect_resource_freq['Resource Type'][i] = resource_typer(defect_resource_freq['Resource'][i])

/Users/willwright/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [75]:
defect_resource_freq[0:10]

,Resource,Frequency,Percent of Wins,Percentile Rank,Rank,Resource Type
0,AscendersBane,1646,0.986219,0.002488,1.0,card
1,Dualcast,1513,0.906531,0.004975,2.0,card
2,Zap,1401,0.839425,0.007463,3.0,card
3,Cracked Core,1394,0.835231,0.009950,4.0,relic
4,Coolheaded,1357,0.813062,0.012438,5.0,card
5,Defragment,1163,0.696824,0.014925,6.0,card
6,Hologram,1141,0.683643,0.017413,7.0,card
7,Glacier,1039,0.622528,0.019900,8.0,card
8,Cold Snap,931,0.557819,0.022388,9.0,card
9,Capacitor,889,0.532654,0.024876,10.0,card


Now, we simply subset on 'Resource Type' and re-apply the rank and percentile:

In [81]:
# cards
defect_card_freq = copy.copy(defect_resource_freq[defect_resource_freq['Resource Type']=='card'])
del defect_card_freq['Resource Type'] # redundant column since this is only cards
defect_card_freq['Percentile Rank']=defect_card_freq['Frequency'].rank(pct=True, ascending = False)
defect_card_freq['Rank']=defect_card_freq['Frequency'].rank(ascending = False)

# relics
defect_relic_freq = copy.copy(defect_resource_freq[defect_resource_freq['Resource Type']=='relic'])
del defect_relic_freq['Resource Type'] # redundant column since this is only cards
defect_relic_freq['Percentile Rank']=defect_relic_freq['Frequency'].rank(pct=True, ascending = False)
defect_relic_freq['Rank']=defect_relic_freq['Frequency'].rank(ascending = False)

With the framework complete for Defect, we can simply apply the same methodology to Silent and Ironclad:

#### Silent Resource Table and Summary Frequency Tables

In [98]:
# Create Resource Table
silent_asc20_win_resources = resource_table_generator(unique_cards+unique_relics, silent_asc20_win_games)
silent_resource_freq = pd.DataFrame({'Resource':silent_asc20_win_resources['Resource'],
                                     'Frequency':silent_asc20_win_resources.sum(axis = 1)})
silent_resource_freq = silent_resource_freq.sort_values(by = ['Frequency'], ascending = False).reset_index(drop=True)
silent_resource_freq['Percent of Wins'] = silent_resource_freq['Frequency']/len(silent_asc20_win_games)

In [99]:
# subset to resources which are >0
silent_resource_freq = silent_resource_freq[silent_resource_freq['Frequency']>0]
silent_resource_freq['Percentile Rank']=silent_resource_freq['Frequency'].rank(pct=True, ascending = False)
silent_resource_freq['Rank']=silent_resource_freq['Frequency'].rank(ascending = False)

# add resource type
silent_resource_freq['Resource Type']=''
for i in range(len(silent_resource_freq)):
    silent_resource_freq['Resource Type'][i] = resource_typer(silent_resource_freq['Resource'][i])

/Users/willwright/.local/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [100]:
# create separate card and relic tables
# cards
silent_card_freq = copy.copy(silent_resource_freq[silent_resource_freq['Resource Type']=='card'])
del silent_card_freq['Resource Type'] # redundant column since this is only cards
silent_card_freq['Percentile Rank']=silent_card_freq['Frequency'].rank(pct=True, ascending = False)
silent_card_freq['Rank']=silent_card_freq['Frequency'].rank(ascending = False)

# relics
silent_relic_freq = copy.copy(silent_resource_freq[silent_resource_freq['Resource Type']=='relic'])
del silent_relic_freq['Resource Type'] # redundant column since this is only cards
silent_relic_freq['Percentile Rank']=silent_relic_freq['Frequency'].rank(pct=True, ascending = False)
silent_relic_freq['Rank']=silent_relic_freq['Frequency'].rank(ascending = False)


#### Ironclad Resource Table and Summary Frequency Tables

In [90]:
# Create Resource Table
ironclad_asc20_win_resources = resource_table_generator(unique_cards+unique_relics, ironclad_asc20_win_games)
ironclad_resource_freq = pd.DataFrame({'Resource':ironclad_asc20_win_resources['Resource'],
                                     'Frequency':ironclad_asc20_win_resources.sum(axis = 1)})
ironclad_resource_freq = ironclad_resource_freq.sort_values(by = ['Frequency'], ascending = False).reset_index(drop=True)
ironclad_resource_freq['Percent of Wins'] = ironclad_resource_freq['Frequency']/len(ironclad_asc20_win_games)

In [93]:
# subset to resources which are >0
ironclad_resource_freq = ironclad_resource_freq[ironclad_resource_freq['Frequency']>0]
ironclad_resource_freq['Percentile Rank']=ironclad_resource_freq['Frequency'].rank(pct=True, ascending = False)
ironclad_resource_freq['Rank']=ironclad_resource_freq['Frequency'].rank(ascending = False)

# add resource type
ironclad_resource_freq['Resource Type']=''
for i in range(len(ironclad_resource_freq)):
    ironclad_resource_freq['Resource Type'][i] = resource_typer(ironclad_resource_freq['Resource'][i])

/Users/willwright/.local/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [94]:
# create separate card and relic tables
# cards
ironclad_card_freq = copy.copy(ironclad_resource_freq[ironclad_resource_freq['Resource Type']=='card'])
del ironclad_card_freq['Resource Type'] # redundant column since this is only cards
ironclad_card_freq['Percentile Rank']=ironclad_card_freq['Frequency'].rank(pct=True, ascending = False)
ironclad_card_freq['Rank']=ironclad_card_freq['Frequency'].rank(ascending = False)

# relics
ironclad_relic_freq = copy.copy(ironclad_resource_freq[ironclad_resource_freq['Resource Type']=='relic'])
del ironclad_relic_freq['Resource Type'] # redundant column since this is only cards
ironclad_relic_freq['Percentile Rank']=ironclad_relic_freq['Frequency'].rank(pct=True, ascending = False)
ironclad_relic_freq['Rank']=ironclad_relic_freq['Frequency'].rank(ascending = False)


#### Saving Frequency Table Results

In [101]:
# Write to .csv
# Defect
defect_resource_freq.to_csv('../results/defect_resource_freq.csv', index = False)
defect_card_freq.to_csv('../results/defect_card_freq.csv', index = False)
defect_relic_freq.to_csv('../results/defect_relic_freq.csv', index = False)

# Silent
silent_resource_freq.to_csv('../results/silent_resource_freq.csv', index = False)
silent_card_freq.to_csv('../results/silent_card_freq.csv', index = False)
silent_relic_freq.to_csv('../results/silent_relic_freq.csv', index = False)

# Ironclad
ironclad_resource_freq.to_csv('../results/ironclad_resource_freq.csv', index = False)
ironclad_card_freq.to_csv('../results/ironclad_card_freq.csv', index = False)
ironclad_relic_freq.to_csv('../results/ironclad_relic_freq.csv', index = False)

___
## Clustering the data
